# OPENAI-NOTEBOOK3  AUTOMATIC EXAM CREATOR

In [1]:
import os
import openai

In [23]:
#os.environ['OPEN_AI_KEY'] = ''
openai.api_key = os.getenv('OPEN_AI_KEY')

## Prompt Design

In [26]:
# Create prompt for fake company
prompt = 'Give me details about the technology start-up mimi and pimo'

In [27]:
# Set up response with the prompt
response = openai.Completion.create(engine = 'text-davinci-003', 
                                    prompt = prompt, 
                                    max_tokens = 150, 
                                    temperature = 0.6)

In [28]:
# Print the response detailing the fake company
print(response['choices'][0]['text'])



Mimi and Pimo is a technology start-up company that creates intelligent technology to help people live healthier and more sustainable lifestyles. Founded in 2019, the company is based in Los Angeles, California. The company's mission is to create technology that helps people make better decisions about their health and the environment. The company focuses on developing products that are easy to use and that provide meaningful insights about people's health and the environment. The company's flagship product is the Mimi and Pimo app, which helps people track their physical activity, nutrition, and environmental impact. The app allows users to set goals, track their progress, and get personalized advice on how to improve their health and the environment. Additionally, the company offers a range of


This is model hallucination

In [ ]:
# Create prompt for fake company with a requirement for the model to be sure it exists
prompt = 'Give me details about the technology start-up mimi and pimo. Only answer if you are 100% sure company exists, otherwise answer dont know'

In [ ]:
# Set up response with the prompt
response = openai.Completion.create(engine = 'text-davinci-003', 
                                    prompt = prompt, 
                                    max_tokens = 150, 
                                    temperature = 0.7)

In [ ]:
# Print the response detailing the fake company
print(response['choices'][0]['text'])

This returns different information but still not correct

## Create Quiz Prompt

In [44]:
# Create the test prompt
def create_test_prompt(topic, num_questions, num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions} questions. " \
    + f"Each question should have {num_possible_answers} options. "\
    + f"Also include the correct answer for each question using the starting string 'Correct Answer: '."
    return prompt

In [45]:
create_test_prompt("python", 4, 4)

"Create a multiple choice quiz on the topic of python consisting of 4 questions. Each question should have 4 options. Also include the correct answer for each question using the starting string 'Correct Answer: '."

In [52]:
prompt = create_test_prompt("australian history", 4, 4)

In [53]:
# Set up response with the prompt
response = openai.Completion.create(engine = 'text-davinci-003', 
                                    prompt = prompt, 
                                    max_tokens = 250, 
                                    temperature = 0.7)

In [54]:
# Print the response 
print(response['choices'][0]['text'])



Question 1: When did Australia become a Commonwealth?
A. 1788 
B. 1901 
C. 1776 
D. 1803

Correct Answer: B. 1901

Question 2: What is the name of Australia's first Prime Minister?
A. John Howard
B. Robert Menzies
C. Edmund Barton
D. Gough Whitlam

Correct Answer: C. Edmund Barton

Question 3: What year did the first European settlement in Australia take place?
A. 1804
B. 1788
C. 1606
D. 1776

Correct Answer: B. 1788

Question 4: What year did Australia become a republic?
A. 1901
B. 1986
C. 1947
D. 1999

Correct Answer: D. 1999


## Q & A Extraction

In [55]:
response['choices'][0]['text']

"\n\nQuestion 1: When did Australia become a Commonwealth?\nA. 1788 \nB. 1901 \nC. 1776 \nD. 1803\n\nCorrect Answer: B. 1901\n\nQuestion 2: What is the name of Australia's first Prime Minister?\nA. John Howard\nB. Robert Menzies\nC. Edmund Barton\nD. Gough Whitlam\n\nCorrect Answer: C. Edmund Barton\n\nQuestion 3: What year did the first European settlement in Australia take place?\nA. 1804\nB. 1788\nC. 1606\nD. 1776\n\nCorrect Answer: B. 1788\n\nQuestion 4: What year did Australia become a republic?\nA. 1901\nB. 1986\nC. 1947\nD. 1999\n\nCorrect Answer: D. 1999"

The response has newline characters which we can take advantage of

In [58]:
# Function to extract the questions for the students only without the answers
def create_student_view(test, num_questions):
    student_view = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer"):
            student_view[question_number] +=line+'\n'
        else:
            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ''
    return student_view      

In [60]:
result = create_student_view(response['choices'][0]['text'],4)
result

{1: '\n\nQuestion 1: When did Australia become a Commonwealth?\nA. 1788 \nB. 1901 \nC. 1776 \nD. 1803\n\n',
 2: "\nQuestion 2: What is the name of Australia's first Prime Minister?\nA. John Howard\nB. Robert Menzies\nC. Edmund Barton\nD. Gough Whitlam\n\n",
 3: '\nQuestion 3: What year did the first European settlement in Australia take place?\nA. 1804\nB. 1788\nC. 1606\nD. 1776\n\n',
 4: '\nQuestion 4: What year did Australia become a republic?\nA. 1901\nB. 1986\nC. 1947\nD. 1999\n\n'}

In [61]:
for key in result:
    print(result[key])



Question 1: When did Australia become a Commonwealth?
A. 1788 
B. 1901 
C. 1776 
D. 1803



Question 2: What is the name of Australia's first Prime Minister?
A. John Howard
B. Robert Menzies
C. Edmund Barton
D. Gough Whitlam



Question 3: What year did the first European settlement in Australia take place?
A. 1804
B. 1788
C. 1606
D. 1776



Question 4: What year did Australia become a republic?
A. 1901
B. 1986
C. 1947
D. 1999




In [69]:
# Function to get the answers only
def extract_answers(test, num_questions):
    answers = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer"):
            answers[question_number] +=line+'\n'
            
            if question_number < num_questions:
                question_number+=1
                answers[question_number] = ''
                
    return answers     

In [70]:
extract_answers(response['choices'][0]['text'],4)

{1: 'Correct Answer: B. 1901\n',
 2: 'Correct Answer: C. Edmund Barton\n',
 3: 'Correct Answer: B. 1788\n',
 4: 'Correct Answer: D. 1999\n'}

## Create the Exam with Grading

In [104]:
student_view = create_student_view(response['choices'][0]['text'],4)
correct_answer = extract_answers(response['choices'][0]['text'],4)

In [105]:
# Function to take this exam

def take(student_view):
    student_answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        student_answers[question] = answer
    return student_answers
        

In [106]:
student_answers = take(student_view)



Question 1: When did Australia become a Commonwealth?
A. 1788 
B. 1901 
C. 1776 
D. 1803


Enter your answer: A

Question 2: What is the name of Australia's first Prime Minister?
A. John Howard
B. Robert Menzies
C. Edmund Barton
D. Gough Whitlam


Enter your answer: d

Question 3: What year did the first European settlement in Australia take place?
A. 1804
B. 1788
C. 1606
D. 1776


Enter your answer: d

Question 4: What year did Australia become a republic?
A. 1901
B. 1986
C. 1947
D. 1999


Enter your answer: a


In [110]:
# Function to compare answers to actual answers

def grade(correct_answer, student_answers):
    correct_answers = 0
    for question, answer in student_answers.items():
        if answer.upper() == correct_answer[question].upper()[16]:
            correct_answers+=1
    grade = 100 * correct_answers/len(answers)
    
    if grade < 60:
        passed = "You did not pass"
    else:
        passed = "You passed!"
        
    return f"{correct_answers}/{len(answers)} correct! You got {grade} grade, {passed}"
            

In [109]:
grade(answers, student_answers)

'0/4 correct! You got 0.0 grade, You did not pass'